In [1]:
#import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import requests
import collections
import re
import json
import glob
import math
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pandas.io.json import json_normalize
from scipy.spatial.distance import cdist, pdist, squareform
import pandas.tseries.holiday as hol
from sklearn.linear_model import LinearRegression
from pygam import LinearGAM, LogisticGAM, s, f
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
# Define tripdf: raw trip history data 
# covering 24 months from July 2017-July 2019
tripdf=pd.DataFrame()
for filename in glob.glob("../Data_raw/Biketrips/*tripdata.csv"):
    tripdf=tripdf.append(pd.read_csv(filename, parse_dates=["starttime", "stoptime"]))

# Clean tripdf column names
tripdf.columns = tripdf.columns.str.replace(' ', '_')

# Take the columns of interest
trip_simp=tripdf[["start_station_id", "end_station_id", "starttime", "stoptime", 
                  "tripduration", "bikeid"]]
# add counting column for future aggregation 
trip_simp["counting"] = 1

/Users/kkim14/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
# Read in station_with_zipzone.csv
station_zip = pd.read_csv("../Data_processed/station_with_zipzone.csv")

In [43]:
# Difference between station IDs from the trip data and from the station data
nonexisting = set(trip_simp.start_station_id.unique()).union(set(trip_simp.end_station_id.unique()))-set(station_zip.ID.unique())
print("There are total "+str(len(nonexisting))+ " stations that no longer exist.")

There are total 55 stations that no longer exist.


In [61]:
# Create a dictionary to clean trip history 
fixdict = { 1:np.nan, 32:np.nan,  44:np.nan,  45:np.nan,  88:381, 155:331, 158:np.nan, 164:np.nan, 168:332, 
           207:np.nan, 220:328, 229:np.nan, 230:np.nan, 231:329, 237:333, 238:334, 240:335, 241:336, 242:337, 
           243:338, 244:339, 245:340, 246:341, 247:342, 248:343, 249:344,250:345, 252:346, 253:347, 
           254:348, 256:349, 257:350, 262:351, 263:353, 264:354, 274:355, 277:356, 278:357, 281:358,
           283:327, 284:359, 286:330, 293:360, 294:361, 295:362, 299:363, 300:364, 302:365, 306:368, 
           307:366, 308:np.nan, 311:367,317:np.nan, 383:np.nan}

In [62]:
# Map former stations to new stations 
# or remove test stations 
trip_simp=trip_simp.replace({"start_station_id":fixdict})
trip_simp=trip_simp.replace({"end_station_id":fixdict})
trip_simp=trip_simp.dropna(subset=["start_station_id"])
trip_simp=trip_simp.dropna(subset=["end_station_id"])

In [12]:
set(station_zip.ID.unique())-set(trip_simp.start_station_id.unique())

{411, 413, 414, 415, 416, 417}

In [66]:
trip_simp.start_station_id=trip_simp.start_station_id.astype(int)
trip_simp.end_station_id=trip_simp.end_station_id.astype(int)

In [68]:
trip_simp.to_csv("../Data_processed/trip_simp.csv")

## tripmonth: aggregation of trip_simp by month
Most of the trips (less than 0.1%) span less than 24 hours. 
We use the start time of each trip. 

In [79]:
shorttrip=trip_simp[trip_simp.tripduration<=86400]
longtrip=trip_simp[trip_simp.tripduration>86400]

print("Only "+'{:.3f}'.format(len(longtrip)/(len(shorttrip)+len(longtrip))*100)
      +"% of the trips last longer than 24 hours.")

Only 0.065% of the trips last longer than 24 hours.


In [80]:
# Wish I knew a better way but this will do
jul2017=trip_simp.set_index("starttime")["2017-07"].groupby("start_station_id").agg({"counting":"sum"})
aug2017=trip_simp.set_index("starttime")["2017-08"].groupby("start_station_id").agg({"counting":"sum"})
sep2017=trip_simp.set_index("starttime")["2017-09"].groupby("start_station_id").agg({"counting":"sum"})
oct2017=trip_simp.set_index("starttime")["2017-10"].groupby("start_station_id").agg({"counting":"sum"})
nov2017=trip_simp.set_index("starttime")["2017-11"].groupby("start_station_id").agg({"counting":"sum"})
dec2017=trip_simp.set_index("starttime")["2017-12"].groupby("start_station_id").agg({"counting":"sum"})
jan2018=trip_simp.set_index("starttime")["2018-01"].groupby("start_station_id").agg({"counting":"sum"})
feb2018=trip_simp.set_index("starttime")["2018-02"].groupby("start_station_id").agg({"counting":"sum"})
mar2018=trip_simp.set_index("starttime")["2018-03"].groupby("start_station_id").agg({"counting":"sum"})
apr2018=trip_simp.set_index("starttime")["2018-04"].groupby("start_station_id").agg({"counting":"sum"})
may2018=trip_simp.set_index("starttime")["2018-05"].groupby("start_station_id").agg({"counting":"sum"})
jun2018=trip_simp.set_index("starttime")["2018-06"].groupby("start_station_id").agg({"counting":"sum"})
jul2018=trip_simp.set_index("starttime")["2018-07"].groupby("start_station_id").agg({"counting":"sum"})
aug2018=trip_simp.set_index("starttime")["2018-08"].groupby("start_station_id").agg({"counting":"sum"})
sep2018=trip_simp.set_index("starttime")["2018-09"].groupby("start_station_id").agg({"counting":"sum"})
oct2018=trip_simp.set_index("starttime")["2018-10"].groupby("start_station_id").agg({"counting":"sum"})
nov2018=trip_simp.set_index("starttime")["2018-11"].groupby("start_station_id").agg({"counting":"sum"})
dec2018=trip_simp.set_index("starttime")["2018-12"].groupby("start_station_id").agg({"counting":"sum"})
jan2019=trip_simp.set_index("starttime")["2019-01"].groupby("start_station_id").agg({"counting":"sum"})
feb2019=trip_simp.set_index("starttime")["2019-02"].groupby("start_station_id").agg({"counting":"sum"})
mar2019=trip_simp.set_index("starttime")["2019-03"].groupby("start_station_id").agg({"counting":"sum"})
apr2019=trip_simp.set_index("starttime")["2019-04"].groupby("start_station_id").agg({"counting":"sum"})
may2019=trip_simp.set_index("starttime")["2019-05"].groupby("start_station_id").agg({"counting":"sum"})
jun2019=trip_simp.set_index("starttime")["2019-06"].groupby("start_station_id").agg({"counting":"sum"})
jul2019=trip_simp.set_index("starttime")["2019-07"].groupby("start_station_id").agg({"counting":"sum"})

# Define tripmonth
tripmonth=pd.concat([jul2017, aug2017, sep2017, oct2017, nov2017, dec2017, 
           jan2018, feb2018, mar2018, apr2018, may2018, jun2018, 
           jul2018, aug2018, sep2018, oct2018, nov2018, dec2018, 
           jan2019, feb2019, mar2019, apr2019, may2019, jun2019, jul2019], axis=1)

tripmonth.columns = ["Jul17", "Aug17", "Sep17", "Oct17", "Nov17", "Dec17", 
           "Jan18", "Feb18", "Mar18", "Apr18", "May18", "Jun18", 
           "Jul18", "Aug18", "Sep18", "Oct18", "Nov18", "Dec18", 
           "Jan19", "Feb19", "Mar19", "Apr19", "May19", "Jun19", "Jul19"]

In [81]:
# Some stations did not operate during winter
tripmonth.head(3)

,Jul17,Aug17,Sep17,Oct17,Nov17,Dec17,Jan18,Feb18,Mar18,Apr18,...,Oct18,Nov18,Dec18,Jan19,Feb19,Mar19,Apr19,May19,Jun19,Jul19
start_station_id,,,,,,,,,,,,,,,,,,,,,
3,602.0,659.0,532.0,522.0,306.0,164.0,168.0,288.0,289.0,287.0,...,657.0,420.0,259.0,193.0,294.0,333.0,698.0,730.0,917.0,1090.0
4,1182.0,1299.0,1156.0,1056.0,722.0,473.0,311.0,400.0,434.0,602.0,...,1049.0,602.0,441.0,369.0,375.0,454.0,861.0,1212.0,1513.0,1671.0
5,965.0,1053.0,953.0,929.0,587.0,127.0,NaN,NaN,NaN,325.0,...,1393.0,796.0,412.0,190.0,NaN,NaN,593.0,1056.0,1249.0,1658.0


In [82]:
# Some stations recently opened
tripmonth.tail(3)

,Jul17,Aug17,Sep17,Oct17,Nov17,Dec17,Jan18,Feb18,Mar18,Apr18,...,Oct18,Nov18,Dec18,Jan19,Feb19,Mar19,Apr19,May19,Jun19,Jul19
start_station_id,,,,,,,,,,,,,,,,,,,,,
409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0
410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


Counting total number of trips per station is biased since some stations has been operating and started operating for shorter. To compensate for this, we count for how many months each station has been open. 


In [86]:
tripmonth["Num_Op_Months"] = tripmonth.iloc[:, :].notna().sum(axis=1)
tripmonth["Ave_Trips_Per_Month"]=tripmonth.iloc[:, :-1].sum(axis=1)/tripmonth.Num_Op_Months

In [90]:
tripmonth.tail()

,Jul17,Aug17,Sep17,Oct17,Nov17,Dec17,Jan18,Feb18,Mar18,Apr18,...,Dec18,Jan19,Feb19,Mar19,Apr19,May19,Jun19,Jul19,Num_Op_Months,Ave_Trips_Per_Month
start_station_id,,,,,,,,,,,,,,,,,,,,,
407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,1,42.0
408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,1,48.0
409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,1,112.0
410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,1.0
412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,4.0


In [91]:
tripmonth.to_csv("../Data_processed/tripmonth.csv")